# LLMs

In [1]:
import os
from langchain.llms import OpenAI

In [2]:
from secret_key import openai_key

In [3]:
os.environ["OPENAI_API_KEY"] = openai_key

In [4]:
llm = OpenAI(temperature=0.9)

In [5]:
name = llm.predict("I want to open a restaurant for malayalam food. Suggest a fancy name for this.")
print(name)




Royal Ushaprasanna Cuisine


# Prompt Templates

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
    )
p = prompt_template_name.format(cuisine = "Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


# Chain

In [9]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt_template_name,verbose = True)
chain.run("arabic")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for arabic food. Suggest a fency name for this.

> Finished chain.


'\n\nMajestic Mirage'

In [10]:
llm = OpenAI(temperature=0.6)
prompt_template_name = PromptTemplate(
        input_variables=['cuisine'],
        template = "i want to open a resturant for {cuisine} food.Suggest a fency name for this."
        )

name_chain = LLMChain(llm=llm,prompt=prompt_template_name)

prompt_template_items = PromptTemplate(input_variables=['resturant_name'],
                                      template="suggest some menu items for {resturant_name}")

food_items_chain = LLMChain(llm=llm,prompt=prompt_template_items)

# Simple Sequential Chain

In [11]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain,food_items_chain])

content = chain.run("Arabic")
print(content)



-Lamb Tagine with Preserved Lemons
-Couscous with Vegetables
-Falafel with Tahini Sauce
-Stuffed Grape Leaves
-Baba Ghanoush
-Roasted Eggplant Salad
-Lentil Soup
-Hummus
-Spicy Lamb Kebabs
-Fattoush Salad
-Baklava


# Sequential Chain

In [12]:
llm = OpenAI(temperature=0.7)

prompt_tenplate_name1  = PromptTemplate(input_variables=["cuisine"],
                                      template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
                                      )
name_chain1 = LLMChain(llm=llm,prompt=prompt_tenplate_name1,output_key="restaurant_name")

In [13]:
prompt_template_items1 = PromptTemplate(input_variables=["restaurant_name"],
                                       template="suggest some menu items for {restaurant_name}."
                                       )
food_items_chain1 = LLMChain(llm=llm,prompt=prompt_template_items1,output_key="menu_items")

In [14]:
from  langchain.chains import SequentialChain

chain1 =SequentialChain(chains = [name_chain1,food_items_chain1],
                       input_variables = ['cuisine'],
                       output_variables = ["restaurant_name","menu_items"]
                       )


In [15]:
chain1({"cuisine" : "indian"})

{'cuisine': 'indian',
 'restaurant_name': "\n\nMaharaja's Cuisine",
 'menu_items': '\n\n1. Vegetable Biryani\n2. Butter Chicken\n3. Dal Makhani\n4. Tandoori Chicken\n5. Paneer Tikka Masala\n6. Naan Bread\n7. Samosa Chaat\n8. Kebabs\n9. Gulab Jamun\n10. Kheer'}

# Agents

In [16]:
from langchain.agents import AgentType,initialize_agent,load_tools
from langchain.llms import OpenAI

In [19]:
tools = load_tools(["wikipedia","llm-math"],llm =llm)

agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose =True
                        )
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should look up Elon Musk to find out when he was born.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. Musk is the founder, chairman, CEO and chief technology officer of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$219 billion as of November 2023, according to the Bloomberg Billionaires Index, and $241 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship thro

'Elon Musk was born on June 28, 1971, making him 51 years old in 2023.'

In [20]:
# main script
import os
from secret_key import serp_key

os.environ['SERPAPI_API_KEY'] = serp_key


In [24]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi","llm-math"],llm = llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True
                        )
agent.run("What was the GDP of US in 2022 plus 5?")




> Entering new AgentExecutor chain...
 I need to find the GDP of US in 2022
Action: Search
Action Input: US GDP in 2022
Observation: $25.46 trillion
Thought: I need to add 5 to this number
Action: Calculator
Action Input: 25.46 + 5
Observation: Answer: 30.46
Thought:

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-0trhoVhwKkubw0Ct6Oatyeuq on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

# Memory

In [25]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Arabic")

In [26]:
print(name)



Al-Mazaya


In [27]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Turkish")
print(name)



Kebab Palace


In [28]:
chain.memory

In [29]:
type(chain.memory)

NoneType

ConversationBuffer Memory

In [37]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm,prompt=prompt_template_name,memory=memory)
name = chain.run("Arabic")
print(name)



Al-Mazaya


In [38]:
name = chain.run("Chinees")
print(name)



Golden Dragon Palace


In [39]:
print(chain.memory.buffer)

Human: Arabic
AI: 

Al-Mazaya
Human: Chinees
AI: 

Golden Dragon Palace


In [40]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Arabic'), AIMessage(content='\n\nAl-Mazaya'), HumanMessage(content='Chinees'), AIMessage(content='\n\nGolden Dragon Palace')]))

# Conversation Chain

In [41]:
from langchain.chains import ConversationChain
convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [42]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [43]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10.'

In [44]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team in the first Cricket World Cup was Clive Lloyd.'

In [45]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  5+5 is equal to 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team in the first Cricket World Cup was Clive Lloyd.


ConversationBufferWindowMemory

In [46]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
 
convo = ConversationChain(llm=OpenAI(temperature=0.7),
                         memory=memory
                         )
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies in 1975. They defeated Australia by 17 runs in the final match.'

In [47]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [48]:
convo.run("Who was the captain ofthe winning team?")

" I'm sorry, I don't know the answer to that question."